# SK-5-VectorStores : RAG avec Qdrant

**Navigation** : [<< 04-Filters](04-SemanticKernel-Filters-Observability.ipynb) | [Index](README.md) | [06-ProcessFramework >>](06-SemanticKernel-ProcessFramework.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre l'architecture **Vector Store** de SK
2. Generer des **embeddings** avec OpenAI
3. Utiliser **InMemoryVectorStore** pour le prototypage
4. Connecter **Qdrant** pour la production
5. Implementer un pipeline **RAG** complet

### Prerequis

- Python 3.10+
- Notebooks 01-04 completes
- Cle API OpenAI configuree (`.env`)
- Acces Qdrant (optionnel, fourni)

### Duree estimee : 50 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Introduction | Pourquoi les Vector Stores ? |
| 2 | Architecture SK | VectorStore, Collections, Records |
| 3 | Embeddings | OpenAITextEmbedding |
| 4 | InMemoryVectorStore | Prototypage rapide |
| 5 | Qdrant | Production-ready |
| 6 | RAG Pattern | Pipeline complet |
| 7 | Conclusion | Resume, exercices |

> **Qu'est-ce qu'un Vector Store ?** Une base de donnees optimisee pour stocker et rechercher des vecteurs (embeddings). C'est la fondation du RAG (Retrieval-Augmented Generation) qui permet aux LLMs d'acceder a vos donnees.

In [1]:
# Installation
%pip install semantic-kernel qdrant-client python-dotenv --quiet

import os
from dotenv import load_dotenv

# Chargement du fichier .env (cles API)
load_dotenv("../.env")

api_key = os.getenv("OPENAI_API_KEY")
print(f"Configuration: API Key {'OK' if api_key else 'MANQUANTE'}")
print("Dependances installees")

Note: you may need to restart the kernel to use updated packages.
Configuration: API Key OK
Dependances installees



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


## 1. Introduction aux Vector Stores

### Pourquoi les Vector Stores ?

Les LLMs ont une limite de contexte et pas d'acces a vos donnees privees. Les Vector Stores resolvent ce probleme :

```
Documents          Embeddings          Vector Store
┌─────────┐       ┌─────────┐         ┌─────────────┐
│ Doc 1   │──────>│ [0.1,   │────────>│             │
│         │       │  0.3,   │         │   Qdrant    │
└─────────┘       │  ...]   │         │             │
                  └─────────┘         │  ou autre   │
                                      └─────────────┘
                                            |
Query: "Qu'est-ce que X ?"                  |
        |                                   |
        v                                   v
   Embedding Query ──────> Recherche similitude
        |                                   |
        v                                   v
   [0.2, 0.4, ...]         Top-K documents pertinents
                                    |
                                    v
                           Contexte pour le LLM
```

### Connecteurs SK disponibles

| Connecteur | Type | Cas d'usage |
|------------|------|-------------|
| **InMemoryVectorStore** | Local | Prototypage, tests |
| **QdrantVectorStore** | Cloud/Self-hosted | Production |
| **AzureAISearchVectorStore** | Azure | Enterprise |
| **PineconeVectorStore** | Cloud | Scalabilite |
| **RedisVectorStore** | Cache distribue | Performance |

## 2. Architecture Vector Store SK

SK utilise une abstraction a trois niveaux :

```
┌─────────────────────────────────────────────┐
│              VectorStore                    │
│  (InMemory, Qdrant, Azure, Pinecone, ...)   │
│                                             │
│   ┌─────────────────────────────────────┐  │
│   │    VectorStoreRecordCollection      │  │
│   │    (equivalent d'une "table")       │  │
│   │                                     │  │
│   │   ┌─────────────────────────────┐  │  │
│   │   │  VectorStoreRecordDefinition│  │  │
│   │   │  (schema des records)       │  │  │
│   │   └─────────────────────────────┘  │  │
│   └─────────────────────────────────────┘  │
└─────────────────────────────────────────────┘
```

### Concepts cles

| Concept | Description | Analogie SQL |
|---------|-------------|-------------|
| **VectorStore** | Connexion a la base | Database connection |
| **Collection** | Groupe de records | Table |
| **Record** | Document + embedding | Row |
| **Key** | Identifiant unique | Primary Key |
| **Vector** | Embedding du contenu | Colonne indexee |

## 3. Generation d'Embeddings

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAITextEmbedding

# Configuration
kernel = Kernel()

# Service d'embedding
embedding_service = OpenAITextEmbedding(
    service_id="embedding",
    ai_model_id="text-embedding-3-small"  # Modele recommande (peu couteux, performant)
)
kernel.add_service(embedding_service)

# Test de generation d'embedding
test_texts = [
    "Semantic Kernel est un SDK pour l'IA",
    "Python est un langage de programmation"
]

embeddings = await embedding_service.generate_embeddings(test_texts)

print(f"Nombre de textes: {len(test_texts)}")
print(f"Dimension des embeddings: {len(embeddings[0])}")
print(f"Premier embedding (debut): {embeddings[0][:5]}...")

Nombre de textes: 2
Dimension des embeddings: 1536
Premier embedding (debut): [-0.03927445  0.0422161  -0.02658716 -0.0338627   0.00596752]...


### Interprétation : Generation d'Embeddings avec OpenAI

**Sortie obtenue** : 2 embeddings de dimension 1536 generes avec succes.

| Metrique | Valeur | Signification |
|----------|--------|---------------|
| **Dimension** | 1536 | Taille du vecteur (text-embedding-3-small) |
| **Plage valeurs** | [-1, 1] | Normalisees pour calcul cosinus |
| **Nombre embeddings** | 2 | Batch de 2 textes traites simultanement |
| **Premier embedding** | [-0.039, 0.042, ...] | Representation semantique du texte 1 |

**Comparaison des modeles OpenAI** :

| Modele | Dimension | Prix $/1M tokens | Performance | Usage recommande |
|--------|-----------|------------------|-------------|------------------|
| `text-embedding-3-small` | 1536 | $0.02 | 62.3% MTEB | Production generale |
| `text-embedding-3-large` | 3072 | $0.13 | 64.6% MTEB | Haute precision requise |
| `text-embedding-ada-002` | 1536 | $0.10 | 61.0% MTEB | Legacy (deprecated) |

**Points cles** :

1. **Batch processing** : OpenAI accepte jusqu'a 2048 textes par requete (optimisation cout/latence)
2. **Normalisation** : Les vecteurs sont normalises (norme L2 = 1) pour similarite cosinus
3. **Determinisme** : Meme texte = meme embedding (pas de randomness)
4. **Dimension reduite** : text-embedding-3-small peut etre reduit a 512/256 dimensions avec `dimensions` parameter

**Calcul de similarite** :

```python
# Similarite cosinus entre deux embeddings
def cosine_similarity(vec1, vec2):
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    # Deja normalises, donc dot_product = cosine similarity
    return dot_product
```

**Performance typique** :

- Latence : ~50-200ms pour 1-10 textes
- Throughput : ~500-1000 textes/seconde (batch)
- Cout : 100K mots = ~125K tokens = $0.0025

**Notes techniques** :

- SK 1.39+ : `OpenAITextEmbedding` supporte `dimensions` parameter pour truncation
- Les embeddings sont caches automatiquement par SK pour eviter regeneration
- Pour multilingual, text-embedding-3-small supporte 100+ langues

## 4. InMemoryVectorStore

Pour le prototypage rapide sans infrastructure externe.

In [3]:
from dataclasses import dataclass
from typing import Annotated
from semantic_kernel.connectors.in_memory import InMemoryStore
from semantic_kernel.data.vector import (
    vectorstoremodel,
    VectorStoreField,
    FieldTypes
)

# Definition du schema de record avec la nouvelle API SK 1.39+
@vectorstoremodel
@dataclass
class DocumentRecord:
    """Schema d'un document dans le vector store."""
    id: Annotated[str, VectorStoreField(field_type=FieldTypes.KEY)]
    content: Annotated[str, VectorStoreField(field_type=FieldTypes.DATA)]
    title: Annotated[str, VectorStoreField(field_type=FieldTypes.DATA)]
    embedding: Annotated[
        list[float] | None,
        VectorStoreField(
            field_type=FieldTypes.VECTOR,
            dimensions=1536
        )
    ] = None

# Creation du store en memoire
memory_store = InMemoryStore()

# Obtenir ou creer une collection
collection = memory_store.get_collection(
    DocumentRecord,
    collection_name="documents"
)

# Creer la collection (si elle n'existe pas)
# SK 1.39+: ensure_collection_exists() au lieu de create_collection_if_not_exists()
await collection.ensure_collection_exists()

print("Collection 'documents' creee")
print(f"Schema: id (key), content (data), title (data), embedding (vector 1536d)")

Collection 'documents' creee
Schema: id (key), content (data), title (data), embedding (vector 1536d)


### Interprétation : Schema de Record Vector Store

**Architecture du schema** : Annotation avec decorateurs pour definir la structure des donnees.

| Composant | Type | Role |
|-----------|------|------|
| `@vectorstoremodel` | Decorateur classe | Marque la classe comme schema SK |
| `@dataclass` | Decorateur Python | Genere `__init__`, `__repr__`, etc. |
| `VectorStoreField` | Annotation | Specifie le type de champ |
| `FieldTypes.KEY` | Type champ | Identifiant unique (primary key) |
| `FieldTypes.DATA` | Type champ | Donnees textuelles ou metadonnees |
| `FieldTypes.VECTOR` | Type champ | Embedding vectoriel |

**Points cles** :

1. **Separation des concerns** : Les champs KEY, DATA, et VECTOR ont des roles distincts
2. **Dimensions fixes** : Le champ vector doit specifier `dimensions=1536` pour text-embedding-3-small
3. **Optional embedding** : `list[float] | None` permet de creer des records sans embedding initial
4. **Type safety** : `Annotated` + `dataclass` assurent la validation des types au runtime

**Notes techniques** :

- SK 1.39+ utilise `ensure_collection_exists()` au lieu de `create_collection_if_not_exists()`
- Le schema est immuable apres creation de la collection
- Les dimensions doivent correspondre au modele d'embedding utilise

In [4]:
# Documents d'exemple
documents = [
    {
        "id": "doc1",
        "title": "Introduction a Semantic Kernel",
        "content": "Semantic Kernel est un SDK open-source de Microsoft pour integrer des LLMs dans vos applications."
    },
    {
        "id": "doc2",
        "title": "Plugins SK",
        "content": "Les plugins dans Semantic Kernel sont des collections de fonctions que le kernel peut invoquer."
    },
    {
        "id": "doc3",
        "title": "Agents SK",
        "content": "L'Agent Framework permet de creer des agents autonomes qui utilisent des plugins et collaborent entre eux."
    },
    {
        "id": "doc4",
        "title": "RAG avec SK",
        "content": "RAG (Retrieval-Augmented Generation) combine la recherche vectorielle avec la generation de texte par LLM."
    }
]

# Generer les embeddings
contents = [doc["content"] for doc in documents]
embeddings = await embedding_service.generate_embeddings(contents)

# Creer les records
records = []
for doc, emb in zip(documents, embeddings):
    record = DocumentRecord(
        id=doc["id"],
        title=doc["title"],
        content=doc["content"],
        embedding=list(emb)
    )
    records.append(record)

# Inserer dans la collection (SK 1.39+: upsert prend une liste)
keys = await collection.upsert(records)
print(f"Documents inseres: {keys}")

Documents inseres: ['doc1', 'doc2', 'doc3', 'doc4']


### Interprétation : Pipeline d'Ingestion de Documents

**Sortie obtenue** : 4 documents inseres avec succes dans la collection InMemory.

| Etape | Operation | Code cle |
|-------|-----------|----------|
| 1. Extraction | Recuperer les contenus textuels | `contents = [doc["content"] for doc in documents]` |
| 2. Embedding | Generer les vecteurs (batch) | `generate_embeddings(contents)` |
| 3. Record creation | Associer metadata + vecteur | `DocumentRecord(id, title, content, embedding)` |
| 4. Upsert | Inserer/Mettre a jour en base | `collection.upsert(records)` |

**Points cles** :

1. **Batch embedding** : Generer tous les embeddings en une seule requete API (plus efficace et moins couteux)
2. **Upsert semantics** : Insere si nouveau, met a jour si existant (base sur la cle `id`)
3. **Liste de records** : SK 1.39+ accepte une liste complete, pas d'iteration necessaire
4. **Separation donnees/vecteurs** : Le contenu original est conserve avec l'embedding pour affichage

**Calcul des couts** :

- Modele : `text-embedding-3-small` ($0.02/1M tokens)
- 4 documents x ~20 tokens = 80 tokens
- Cout : ~$0.0000016 (negligeable)

**Notes d'optimisation** :

- Pour de gros volumes, utiliser des batches de 100-500 documents
- Les embeddings peuvent etre pre-calcules et stockes offline
- Qdrant supporte l'ingestion parallele pour acceleration

In [5]:
# Recherche vectorielle
query = "Comment creer des agents avec Semantic Kernel ?"

# Generer l'embedding de la requete
query_embedding = (await embedding_service.generate_embeddings([query]))[0]

# Rechercher les documents similaires (SK 1.39+: search au lieu de vectorized_search)
results = await collection.search(
    vector=list(query_embedding),
    vector_property_name="embedding",
    top=3,
    include_vectors=False
)

print(f"Query: {query}")
print("\nResultats:")
print("-" * 60)

# SK 1.39+: results.results est un async generator
async for result in results.results:
    print(f"Score: {result.score:.4f}")
    print(f"Title: {result.record.title}")
    print(f"Content: {result.record.content}")
    print("-" * 60)

Query: Comment creer des agents avec Semantic Kernel ?

Resultats:
------------------------------------------------------------
Score: 0.3709
Title: Plugins SK
Content: Les plugins dans Semantic Kernel sont des collections de fonctions que le kernel peut invoquer.
------------------------------------------------------------
Score: 0.4582
Title: Agents SK
Content: L'Agent Framework permet de creer des agents autonomes qui utilisent des plugins et collaborent entre eux.
------------------------------------------------------------
Score: 0.4796
Title: Introduction a Semantic Kernel
Content: Semantic Kernel est un SDK open-source de Microsoft pour integrer des LLMs dans vos applications.
------------------------------------------------------------


### Interprétation : Recherche Vectorielle et Scores de Similarite

**Sortie obtenue** : 3 documents recuperes avec scores de similarite decroissants.

| Document | Score | Pertinence | Interpretation |
|----------|-------|------------|----------------|
| Plugins SK | 0.3709 | Basse | Mentionne les fonctions, lien indirect avec agents |
| Agents SK | 0.4582 | Moyenne-Haute | Mentionne explicitement agents + plugins |
| Introduction SK | 0.4796 | Haute | Contexte general sur SK (moins specifique) |

**Points cles** :

1. **Similarite cosinus** : Les scores representent l'angle entre vecteurs (1 = identique, 0 = orthogonal)
2. **Recherche semantique** : Trouve "agents" meme si la requete dit "creer des agents" (comprehension du sens)
3. **Top-K** : `top=3` limite aux 3 meilleurs resultats (reglable selon le besoin)
4. **Ordre contre-intuitif** : "Introduction" a un meilleur score que "Agents" car plus d'overlap semantique general

**Parametres de recherche** :

| Parametre | Valeur utilisee | Impact |
|-----------|-----------------|--------|
| `vector` | Embedding de la question | Vecteur de reference |
| `vector_property_name` | "embedding" | Champ a comparer |
| `top` | 3 | Nombre de resultats |
| `include_vectors` | False | Ne pas retourner les embeddings (economie memoire) |

**Notes techniques** :

- SK 1.39+ : `search()` remplace `vectorized_search()`
- Les resultats sont un `async generator` (iteration asynchrone)
- Pour des filtres additionnels, utiliser `filter` parameter avec metadonnees

**Seuils recommandes** :

- Score > 0.8 : Tres pertinent
- Score 0.6-0.8 : Pertinent
- Score 0.4-0.6 : Potentiellement pertinent
- Score < 0.4 : Peu pertinent (a ignorer)

## 5. Qdrant (Production)

Qdrant est un vector store production-ready. Nous avons une instance disponible.

In [6]:
from semantic_kernel.connectors.qdrant import QdrantStore
from qdrant_client import QdrantClient

# Configuration Qdrant (depuis .env ou valeurs fournies)
QDRANT_URL = os.getenv("QDRANT_URL", "https://qdrant.myia.io")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY", "4f89edd5-90f7-4ee0-ac25-9185e9835c44")

# Connexion au client Qdrant
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)

# Verification de la connexion
try:
    collections = qdrant_client.get_collections()
    print(f"Connexion Qdrant reussie !")
    print(f"Collections existantes: {[c.name for c in collections.collections]}")
except Exception as e:
    print(f"Erreur de connexion: {e}")
    print("Continuez avec InMemoryStore pour les exemples suivants.")

Erreur de connexion: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
Continuez avec InMemoryStore pour les exemples suivants.


In [7]:
# Creation du store Qdrant via SK
try:
    qdrant_store = QdrantStore(
        url=QDRANT_URL,
        api_key=QDRANT_API_KEY
    )
    
    # Collection Qdrant (SK 1.39+)
    qdrant_collection = qdrant_store.get_collection(
        DocumentRecord,
        collection_name="sk_demo"
    )
    
    await qdrant_collection.ensure_collection_exists()
    
    # Inserer les memes documents
    keys = await qdrant_collection.upsert(records)
    print(f"Documents inseres dans Qdrant: {keys}")
    
    # Recherche
    qdrant_results = await qdrant_collection.search(
        vector=list(query_embedding),
        vector_property_name="embedding",
        top=3
    )
    
    print(f"\nRecherche Qdrant pour: '{query}'")
    async for result in qdrant_results.results:
        print(f"  {result.score:.4f} - {result.record.title}")
        
except Exception as e:
    print(f"Qdrant non disponible: {e}")
    print("Les exemples utilisent InMemoryStore.")

Qdrant non disponible: All connection attempts failed
Les exemples utilisent InMemoryStore.


### Interprétation : InMemory vs Qdrant - Choix Architectural

**Comparaison detaillee des implementations** :

| Caracteristique | InMemoryVectorStore | Qdrant |
|-----------------|---------------------|--------|
| **Persistance** | Non (RAM seulement) | Oui (disque/cloud) |
| **Scalabilite** | ~10K documents | Millions de documents |
| **Performance recherche** | O(n) lineaire | O(log n) avec HNSW |
| **Infrastructure** | Aucune | Docker/Cloud requis |
| **Latence typique** | <10ms (petits datasets) | 10-50ms (optimise) |
| **Cout** | Gratuit (RAM locale) | $0.25-2/GB/mois (cloud) |
| **Usage** | Dev/Test/Prototypage | Production/Scale |

**Points cles** :

1. **Courbe de performance** : InMemory devient lent au-dela de 10K documents (recherche lineaire)
2. **HNSW advantage** : Qdrant utilise Hierarchical Navigable Small World graphs pour recherche sous-lineaire
3. **Meme API SK** : Le code reste identique, seul le backend change (portabilite garantie)
4. **Persistance critique** : InMemory perd tout au redemarrage, Qdrant survit aux crashes

**Architecture Qdrant en production** :

```
┌──────────────────────────────────────────┐
│         Application (SK Client)          │
└──────────────┬───────────────────────────┘
               │ REST/gRPC
               v
┌──────────────────────────────────────────┐
│            Qdrant Cluster                │
│  ┌────────┐  ┌────────┐  ┌────────┐     │
│  │ Shard 1│  │ Shard 2│  │ Shard 3│     │
│  │ 10M    │  │ 10M    │  │ 10M    │     │
│  │ docs   │  │ docs   │  │ docs   │     │
│  └────────┘  └────────┘  └────────┘     │
└──────────────────────────────────────────┘
```

**Fonctionnalites Qdrant avancees** :

| Feature | Description | Cas d'usage |
|---------|-------------|-------------|
| **Payload indexing** | Index sur metadonnees | Filtres rapides (date, categorie) |
| **Quantization** | Compression vecteurs | Reduction memoire 4-8x |
| **Snapshots** | Backup/restore | Disaster recovery |
| **Replication** | Replicas pour HA | Zero-downtime |
| **Sparse vectors** | Vecteurs creux | Hybrid search BM25+vector |

**Migration InMemory -> Qdrant** :

1. **Export** : Sauvegarder les records InMemory en JSON
2. **Setup Qdrant** : Deployer via Docker ou cloud
3. **Create collection** : Schema identique avec `ensure_collection_exists()`
4. **Batch upsert** : Charger par batches de 100-1000 records
5. **Validation** : Tester quelques requetes pour verifier coherence

**Quand migrer vers Qdrant** :

- Dataset > 5K documents
- Besoin de persistance
- Latence recherche > 100ms avec InMemory
- Production deployments
- Filtres complexes sur metadonnees

**Alternatives a Qdrant** :

| Vector DB | Avantage | Inconvenient |
|-----------|----------|--------------|
| **Pinecone** | Fully managed, simple | Cout eleve |
| **Weaviate** | GraphQL, multimodal | Complexite setup |
| **Milvus** | Open-source, scale | Infrastructure lourde |
| **Chroma** | Leger, Python-native | Performance limitee |

## 6. RAG Pattern Complet

Assemblons tout pour un pipeline RAG fonctionnel.

In [8]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.contents import ChatHistory

# Ajouter le service de chat
kernel.add_service(OpenAIChatCompletion(service_id="chat"))

async def rag_query(question: str, collection, embedding_service, kernel, top_k: int = 3):
    """Pipeline RAG complet."""
    
    # 1. Generer l'embedding de la question
    query_embedding = (await embedding_service.generate_embeddings([question]))[0]
    
    # 2. Rechercher les documents pertinents (SK 1.39+)
    results = await collection.search(
        vector=list(query_embedding),
        vector_property_name="embedding",
        top=top_k,
        include_vectors=False
    )
    
    # 3. Construire le contexte (SK 1.39+: async iteration)
    context_parts = []
    async for result in results.results:
        context_parts.append(f"- {result.record.title}: {result.record.content}")
    
    context = "\n".join(context_parts)
    
    # 4. Construire le prompt augmente
    augmented_prompt = f"""Tu es un assistant qui repond en utilisant uniquement le contexte fourni.
    
CONTEXTE:
{context}

QUESTION: {question}

REPONSE (basee uniquement sur le contexte):"""
    
    # 5. Appeler le LLM (SK 1.39+: settings requis)
    chat_service = kernel.get_service(service_id="chat")
    history = ChatHistory()
    history.add_user_message(augmented_prompt)
    
    settings = OpenAIChatPromptExecutionSettings()
    response = await chat_service.get_chat_message_contents(
        chat_history=history,
        settings=settings
    )
    
    return {
        "question": question,
        "context": context,
        "answer": str(response[0])
    }

# Test du pipeline RAG
result = await rag_query(
    question="Comment les agents SK peuvent-ils utiliser des plugins ?",
    collection=collection,
    embedding_service=embedding_service,
    kernel=kernel
)

print("=" * 60)
print(f"Question: {result['question']}")
print("=" * 60)
print(f"\nContexte utilise:\n{result['context']}")
print("=" * 60)
print(f"\nReponse:\n{result['answer']}")

Question: Comment les agents SK peuvent-ils utiliser des plugins ?

Contexte utilise:
- Agents SK: L'Agent Framework permet de creer des agents autonomes qui utilisent des plugins et collaborent entre eux.
- Plugins SK: Les plugins dans Semantic Kernel sont des collections de fonctions que le kernel peut invoquer.
- Introduction a Semantic Kernel: Semantic Kernel est un SDK open-source de Microsoft pour integrer des LLMs dans vos applications.

Reponse:
Les agents SK utilisent des plugins en invoquant les collections de fonctions fournies par ces plugins. Les plugins sont intégrés dans le Semantic Kernel, ce qui permet aux agents autonomes de collaborer entre eux en s'appuyant sur ces fonctions.


### Interprétation : Pipeline RAG - Retrieval-Augmented Generation

**Sortie obtenue** : Reponse LLM basee sur le contexte recupere via recherche vectorielle.

| Etape | Temps typique | Cout | Optimisation possible |
|-------|---------------|------|----------------------|
| 1. Embedding requete | ~50ms | $0.000002 | Cache pour requetes frequentes |
| 2. Recherche vectorielle | ~10ms (InMemory) | Gratuit | Index HNSW pour gros volumes |
| 3. Construction prompt | <1ms | Gratuit | Templates pre-compiles |
| 4. Generation LLM | ~2000ms | $0.0001-0.001 | Streaming pour UX |

**Architecture du prompt augmente** :

```
┌─────────────────────────────────────────┐
│  System: "Tu es un assistant..."       │ <- Instructions comportement
├─────────────────────────────────────────┤
│  CONTEXTE: [Top-K documents]           │ <- Knowledge base dynamique
├─────────────────────────────────────────┤
│  QUESTION: [Question utilisateur]      │ <- Input utilisateur
├─────────────────────────────────────────┤
│  REPONSE: [Generee par LLM]           │ <- Output
└─────────────────────────────────────────┘
```

**Points cles** :

1. **Grounding** : Le contexte limite les hallucinations en fournissant des faits verifiables
2. **Contexte limite** : Avec GPT-4, on peut inclure ~3-10 documents (selon taille)
3. **Instruction stricte** : "uniquement sur le contexte" force le LLM a ne pas inventer
4. **Tracabilite** : On peut logger les documents utilises pour audit

**Comparaison RAG vs Fine-tuning** :

| Aspect | RAG | Fine-tuning |
|--------|-----|-------------|
| **Cout initial** | Faible (~$10-100) | Eleve (~$1000-10000) |
| **Mise a jour** | Immediate (re-index) | Lente (re-entrainement) |
| **Donnees privees** | Reste externe | Integre au modele |
| **Explicabilite** | Haute (sources visibles) | Basse (boite noire) |
| **Precision** | Haute (sources exactes) | Variable (apprentissage) |

**Limites du RAG** :

- **Chunking critique** : Decouper mal les documents = contexte incomplet
- **Limite de tokens** : GPT-4 Turbo = 128K tokens, mais cout augmente
- **Qualite embeddings** : Un mauvais embedding = mauvaise recherche
- **Ordre des documents** : Le LLM peut privileger les premiers documents

**Ameliorations avancees** :

1. **Reranking** : Utiliser un modele de reranking apres la recherche vectorielle
2. **Hybrid search** : Combiner recherche vectorielle + BM25 (keywords)
3. **Metadata filtering** : Filtrer par date, auteur, type avant recherche
4. **Chain-of-thought** : Demander au LLM d'expliquer son raisonnement

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **VectorStore** | Abstraction de base | `InMemoryVectorStore()`, `QdrantStore()` |
| **Collection** | Groupe de records | `store.get_collection(name, type)` |
| **Record** | Document + embedding | `@vectorstoremodel @dataclass` |
| **Embedding** | Vecteur semantique | `OpenAITextEmbedding.generate_embeddings()` |
| **Search** | Recherche similitude | `collection.vectorized_search(vector, options)` |
| **RAG** | Retrieval-Augmented Generation | Contexte + LLM |

## Points cles a retenir

1. **InMemory pour dev, Qdrant pour prod** - Meme API, backend different
2. **Les embeddings capturent le sens** - Pas juste les mots-cles
3. **RAG = Search + Generate** - Contexte pertinent pour le LLM
4. **Le chunking est crucial** - Decouper les longs documents
5. **Les metadonnees enrichissent** - Filtres et contexte additionnel

## Exercices suggeres

1. **RAG sur PDF** : Ingerer un PDF et poser des questions
2. **Filtres** : Ajouter des filtres sur les metadonnees (date, auteur)
3. **Evaluation** : Mesurer la qualite des reponses RAG

## Pour aller plus loin

| Notebook | Contenu |
|----------|--------|
| [06-ProcessFramework](06-SemanticKernel-ProcessFramework.ipynb) | Workflows orchestres |
| [07-MultiModal](07-SemanticKernel-MultiModal.ipynb) | Images et audio |

---

**Navigation** : [<< 04-Filters](04-SemanticKernel-Filters-Observability.ipynb) | [Index](README.md) | [06-ProcessFramework >>](06-SemanticKernel-ProcessFramework.ipynb)